In [1]:
import pandas as pd
import json
import importlib
import datetime
import sys
from sources import earwolf, rss
import contentful_helper as contentful

In [2]:
rss_eps = rss.scrape()
newest_saved_episode = contentful.get_newest_episode()

STARTING RSS
Could not parse number for Comedy Bang! Bang! Presents: We Have To Stop Talking TMNT on CBB
Could not parse number for R U Talkin' R.E.M. RE: ME?
Could not parse number for Bonus: Live from UCB Theatre New York
Could not parse number for Bonus: Live from SF Sketch Fest
Could not parse number for Bonus: Live from Vancouver Comedy Fest
RSS DONE


In [3]:
# Compare newest saved episode to newest from RSS

newest_episode_date = pd.to_datetime(rss_eps["releaseDate"]).max().date()
newest_saved_episode_date = datetime.date.fromisoformat(newest_saved_episode["releaseDate"])

# if newest_episode_date <= newest_saved_episode_date:
#     print("No new episodes")
#     sys.exit(0)

earwolf_eps = earwolf.scrape()

STARTING EARWOLF
Could not parse number for <li>Ep #UB1 - <a href="/episode/bonus-the-mysterious-secrets-of-uncle-berties-botanarium-episode-1/">Bonus! The Mysterious Secrets Of Uncle Bertie’s Botanarium – Episode 1</a> (<span>Jemaine Clement</span>)</li>
Could not parse number for <li>Ep #B3 - <a href="/episode/live-from-ucb-ny/">Live from UCB NY!</a> (<span>Zach Galifianakis</span>, <span>David Cross</span>, <span>Todd Barry</span>, <span>John Gemberling</span> &amp; <span>Ted Leo</span>)</li>
Could not parse number for <li>Ep #B2 - <a href="/episode/live-from-sf-sketchfest/">Live from SF Sketchfest!</a> (<span>Doug Benson</span>, <span>Michael Ian Black</span>, <span>Dana Gould</span>, <span>Paul F. Tompkins</span> &amp; <span>Reggie Watts</span>)</li>
Could not parse number for <li>Ep #B1 - <a href="/episode/live-from-vancouver/">Live from Vancouver!</a> (<span>Todd Barry</span>, <span>Nick Thune</span>, <span>Paul F. Tompkins</span>, <span>Garfunkel &amp; Oates</span> &amp; <span>

In [4]:
# Merge sources

episodes = pd.merge(earwolf_eps, rss_eps, on=["number", "bestOf", "live"], how="inner")
episodes.rename(columns={"guests_x":"guests"}, inplace=True)
del episodes['guests_y']

In [42]:
# Keep episodes newer than the newest saved episode

new_episodes = []
for i, row in episodes.iterrows():
    if datetime.date.fromisoformat(row["releaseDate"]) > newest_saved_episode_date:
        new_episodes.append(row.to_dict())

In [9]:
importlib.reload(contentful)

# For each guest, save only if they are new, and get id

for episode in new_episodes:
    guest_ids = []
    for guest in json.loads(episode["guests"]):
        id = contentful.get_guest(guest)
        if id == None:
            id = contentful.write_guest(guest)["sys"]["id"]
            contentful.publish(id)
        guest_ids.append(id)
    episode["guest_ids"] = guest_ids

In [14]:
# Write new episodes

for episode in new_episodes:
    id = contentful.write_episode(title=episode["title"], number=episode["number"],releaseDate=episode["releaseDate"], guest_ids=episode["guest_ids"], bestOf=episode["bestOf"],earwolfUrl=episode["earwolfUrl"])["sys"]["id"]
    contentful.publish(id)
